### Compute zonal statisitics for Landsat 8 and Senintel 1 within select AOIs.

Author: @developmentseed


In [109]:
import os, glob, fnmatch
import json, geojson
from collections import defaultdict
import rasterio
import numpy as np
import rasterstats
from rasterstats import zonal_stats

from enum import Enum
import numpy
from typing import Iterable

import rioxarray
import xarray
import rioxarray
from rioxarray.merge import merge_arrays
import pandas as pd
import argparse

import matplotlib.pyplot as plt

In [110]:
import os 
import folium
from folium import plugins
from rasterio.warp import calculate_default_transform, reproject, Resampling

#### Forest vs. Non-forest
For demonstration purposes, we have selected two small subset areas in Brazil to sample forest and non-forest probabilities from. You can see them on the map below. We will compute the zonal statistics for these cross sections of the AOI. Optional code for mosaicing and clipping follows.

In [111]:
root_dir = '/Users/lillythomas/Downloads/'

In [112]:
# Create a map using Stamen Terrain, centered on study area with set zoom level
m = folium.Map(location=[-3.73725, -51.59917],
                   zoom_start = 12)

# Overlay raster called img using add_child() function (opacity and bounding box set)
folium.GeoJson(os.path.join(root_dir,'para_forest.geojson'), name="geojson").add_to(m)
folium.GeoJson(os.path.join(root_dir,'para_nonforest.geojson'), name="geojson").add_to(m)


In [113]:
# Display map 
m

#### Now let's process Landsat and Sentinel imagery for the AOI.
We will compute NDVI and reproject our NDVI and terrain corrected SAR image to EPSG:4326.

In [114]:
landsat_scene_ids = ['LC08_L1TP_225063_20171024_20200902_02_T1']

In [115]:
sentinel_scene_ids = ['S1A_IW_20171018T090628_DVP_RTC30_G_gpuned_2111']

In [116]:
def landsat_read(landsat_scene_id):
    
    """
    Function to read select Landsat 8 bands from granule. 
    
    Select bands for NDVI are red (band 4) and near infrared (band 5).
    
    Returns: an image array for each band.
    """
    
    landsat_dir = os.path.join(root_dir,landsat_scene_id)
    
    if not os.path.exists(landsat_dir):
        os.makedirs(landsat_dir)
    
    red_path = os.path.join(root_dir,landsat_scene_id+'_B4.TIF')
    nir_path = os.path.join(root_dir,landsat_scene_id+'_B5.TIF')
    qa_path = os.path.join(root_dir,landsat_scene_id+'_QA_PIXEL.TIF')
    
    xarr_red = rioxarray.open_rasterio(red_path)
    xarr_nir = rioxarray.open_rasterio(nir_path)
    
    
    return landsat_dir, xarr_red, xarr_nir, qa_path

In [117]:
class Collection2_QAValues(Enum):
    """Collection 2 QA band info."""

    DesignatedFill = {"out": 1, "bits": (0, 0), "binary": True}
    DilatedCloud = {"out": 8, "bits": (1, 1), "binary": True}
    Cirrus = {"out": 7, "bits": (2, 2), "binary": True}
    Cloud = {"out": 4, "bits": (3, 3), "binary": True}
    CloudShadow = {"out": 5, "bits": (4, 4), "binary": True}
    SnowIce = {"out": 6, "bits": (5, 5), "binary": True}
    Clear = {"out": 0, "bits": (6, 6), "binary": True}
    Water = {"out": 9, "bits": (7, 7), "binary": True}


def _capture_bits(arr, b1, b2, binary):
    width_int = int((b1 - b2 + 1) * "1", 2)
    out = ((arr >> b2) & width_int).astype("uint8")
    return out == 1 if binary else out == 3


def pixel_to_qa(data: numpy.ndarray, QAValues: Iterable) -> numpy.ndarray:
    """Convert DN to QA."""
    output_data = numpy.zeros(data.shape, dtype=numpy.uint16)
    for qa in QAValues:
        bits = qa.value["bits"]
        out = _capture_bits(data, bits[1], bits[0], qa.value["binary"])
        output_data[out] = qa.value["out"]

    return output_data

In [118]:
def get_clear_qa_mask(qa_path):
    with rioxarray.open_rasterio(qa_path, lock=False, chunks=(1, "auto", -1)) as qa:
        qa = qa.rio.set_nodata(1)
        qa = qa.sel(band=1)
        mask = pixel_to_qa(qa.values, QAValues=Collection2_QAValues)
        return mask == Collection2_QAValues.Clear.value['out']

In [119]:
ndvi_arrays = []

In [120]:
for l8 in landsat_scene_ids:
    landsat_dir, red, nir, qa = landsat_read(l8)
    meta.update(dtype="float32")
    # calculate NDVI from Landsat bands 4 and 5. 
    calc_ndvi = (nir-red)/(nir.astype(np.float32)+red)
    # mask out clouds and get clear pixels
    clear_mask = get_clear_qa_mask(qa)
    # calculate and write NDVI to file.
    ndvi_path = os.path.join(landsat_dir, l8+"_NDVI.tif")
    ndvi = calc_ndvi.where(clear_mask)
    ndvi = ndvi.rio.reproject("EPSG:4326")
    ndvi.rio.to_raster(os.path.join(landsat_dir, l8+"_NDVI_4326.tif"))
    ndvi_arrays.append(ndvi)

In [121]:
sar_arrays = []

In [122]:
for s1 in sentinel_scene_ids:
    sentinel_dir = os.path.join(root_dir,s1)
    sar = rioxarray.open_rasterio(os.path.join(sentinel_dir, s1+"_VV.tif"))
    sar = sar.rio.reproject("EPSG:4326")
    sar.rio.to_raster(os.path.join(sentinel_dir, s1+"_VV_4326.tif"))
    sar_arrays.append(sar)

#### Compute zonal statistics for each subset

In [128]:
forest_distr_sar = zonal_stats(os.path.join(root_dir,'para_forest.geojson'), os.path.join(sentinel_dir, s1+"_VV_4326.tif"),
            stats="std min mean max")

nonforest_distr_sar = zonal_stats(os.path.join(root_dir,'para_nonforest.geojson'), os.path.join(sentinel_dir, s1+"_VV_4326.tif"),
            stats="std min mean max")   

forest_distr_ndvi = zonal_stats(os.path.join(root_dir,'para_forest.geojson'), os.path.join(landsat_dir, l8+"_NDVI_4326.tif"),
            stats="std min mean max")

nonforest_distr_ndvi = zonal_stats(os.path.join(root_dir,'para_nonforest.geojson'), os.path.join(landsat_dir, l8+"_NDVI_4326.tif"),
            stats="std min mean max")

In [129]:
forest_distr_sar, nonforest_distr_sar, forest_distr_ndvi, nonforest_distr_ndvi

([{'std': None, 'min': None, 'mean': None, 'max': None},
  {'std': None, 'min': None, 'mean': None, 'max': None},
  {'min': 0.08856558799743652,
   'max': 0.3436959385871887,
   'mean': 0.2028768058067481,
   'std': 0.042478119925853326}],
 [{'std': None, 'min': None, 'mean': None, 'max': None},
  {'std': None, 'min': None, 'mean': None, 'max': None},
  {'min': 0.05659851059317589,
   'max': 0.20904742181301117,
   'mean': 0.11591537522826771,
   'std': 0.029560445353251683}],
 [{'std': None, 'min': None, 'mean': None, 'max': None},
  {'std': None, 'min': None, 'mean': None, 'max': None},
  {'min': 0.38964298367500305,
   'max': 0.47777092456817627,
   'mean': 0.434105220234095,
   'std': 0.0174169704314475}],
 [{'std': None, 'min': None, 'mean': None, 'max': None},
  {'std': None, 'min': None, 'mean': None, 'max': None},
  {'min': 0.17727190256118774,
   'max': 0.35812681913375854,
   'mean': 0.2538168907165527,
   'std': 0.02253473465528542}])

#### Optional processing into clipped mosaics if one image frame doesn't cover your forest and non-forest sample areas

In [26]:
# Function to mosaic L8 and S1 scenes for a single date to cover an AOI    
merged_ndvi = merge_arrays(ndvi_arrays)
merged_sar = merge_arrays(sar_arrays)

In [27]:
# Function to rectify the offset in projection between Sentinel 1 and Landsat. 
# Uses Landsat as the reference image.
merged_sar_repr_match = merged_sar.rio.reproject_match(merged_ndvi)

In [28]:
# Brazil AOI

geojson = '../data/smallAOIs/para_brazil_terrabrasilia.geojson'
with open(geojson) as f:
    data = json.load(f)
    for feature in data['features']:
        geometry = feature['geometry']
        

In [29]:
geometry

{'type': 'Polygon',
 'coordinates': [[[-53.525390625, -5.637852598770853],
   [-50.64697265624999, -5.637852598770853],
   [-50.64697265624999, -2.5809138477251743],
   [-53.525390625, -2.5809138477251743],
   [-53.525390625, -5.637852598770853]]]}

In [30]:
# Function to clip the mosaics by an AOI geometry and write the images to file with the date suffixed in the filename.
clipped_sar = merged_sar.rio.clip([geometry], "EPSG:4326") 
clipped_ndvi = merged_ndvi.rio.clip([geometry], "EPSG:4326") 

In [38]:
date_landsat = landsat_scene_ids[0][17:25]
date_sentinel = sentinel_scene_ids[0][7:15]

In [ ]:
# write images to file
clipped_ndvi.rio.to_raster(os.path.join(landsat_dir,f"merged_clipped_ndvi_{date_landsat}.tif"))
clipped_sar.rio.to_raster(os.path.join(sentinel_dir,f"merged_clipped_sar_{date_sentinel}.tif"))

#### Create time series from clipped mosaics

In [47]:
clipped_ndvi_1 = clipped_ndvi.copy() # copy the existing array for demo purposes
clipped_sar_1 = clipped_sar.copy() # copy the existing array for demo purposes

In [48]:
# print the dates we are working with for this demo
date_landsat,date_sentinel

('20170517', '20170515')

In [49]:
list_dates_landsat = [20170517, 20170517] # same for demo purposes, normally would be a list of all of the dates available
list_dates_sentinel = [20170515, 20170515] # same for demo purposes, normally would be a list of all of the dates available

# in reality, use a for loop to get all of the dates from the filenames:

"""
for f in landsat_scene_ids:
    date_landsat = f[0][17:25]
    list_dates_landsat.append(date_landsat)

    
for f in sentinel_scene_ids:
    date_sentinel = f[0][7:15]
    list_dates_sentinel.append(date_sentinel)
"""

In [50]:
clipped_ndvi_concat = xarray.concat([clipped_ndvi, clipped_ndvi_1], dim=pd.Index(list_dates_landsat, name='time'))
clipped_sar_concat = xarray.concat([clipped_sar, clipped_sar_1], dim=pd.Index(list_dates_sentinel, name='time'))


In [51]:
clipped_ndvi_concat.shape, clipped_sar_concat.shape

((2, 1, 10361, 7285), (2, 1, 10533, 9201))

#### Plot one of the SAR mosaics on our map

In [ ]:
folium.raster_layers.ImageOverlay(merged_sar[0], 
                                  opacity=1, 
                                  bounds = [[-5.64271709, -53.137940554], [-2.782083644, -50.645837307]]).add_to(m)

In [ ]:
# Display map 
m